In [1]:
import pandas as pd
import numpy as np
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import keras_nlp
import platform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasNLP Version: {keras_nlp.__version__}")
print(f"Python Version: {platform.python_version()}")

# Recommended: TensorFlow >= 2.15, KerasNLP >= 0.8

2025-11-27 12:36:39.045002: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 12:36:39.062461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-27 12:36:39.178022: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-27 12:36:39.273566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764218199.342076    2203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764218199.36

TensorFlow Version: 2.19.1
KerasNLP Version: 0.23.0
Python Version: 3.12.3


In [2]:
# Check for GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Detected:", gpus)
    # If the GPU is detected and TensorFlow is configured correctly,
    # cuDNN is almost certainly working and enabled.
    print("cuDNN status: Inferred as ENABLED and operational.")
else:
    print("No GPU detected. cuDNN is not being used.")

GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
cuDNN status: Inferred as ENABLED and operational.


W0000 00:00:1764169238.844279     811 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [12]:
df = pd.read_csv('data/csv_training/training_course_directory.csv')
df.sample(n=2)

,coursetitle,trainingprovideralias,ihl_status,courseratings_stars,jobcareer_impact_stars,full_course_fee,course_fee_after_subsidies,number_of_hours,training_commitment,about_this_course,what_you_learn,attendancecount
20277,Advanced Certificate in Supply Chain Intellige...,SINGAPORE MANAGEMENT UNIVERSITY,IHL,0.0,0.0,3000.0,900.0,22.0,Full Time,Analysing the supply chain is of paramount imp...,"At the end of the 3-day module, participants w...",0.0
12321,Network Analysis and Forensics,Singapore Polytechnic,IHL,0.0,0.0,1824.0,1824.0,60.0,Part Time,This course aims to equip diploma (and above) ...,The course introduces students with the fundam...,0.0


In [13]:
df.columns.tolist()

['coursetitle',
 'trainingprovideralias',
 'ihl_status',
 'courseratings_stars',
 'jobcareer_impact_stars',
 'full_course_fee',
 'course_fee_after_subsidies',
 'number_of_hours',
 'training_commitment',
 'about_this_course',
 'what_you_learn',
 'attendancecount']

In [16]:
# Combine all descriptive columns (A-E) into a single text feature
# This embeds the structured data (Rating, Fee, etc.) directly into the text for BERT
df['Full_Text'] = (
    "TITLE: " + df['coursetitle'].astype(str) + ". " +
    "SCHOOL: " + df['ihl_status'].astype(str) + ". " +
    "COURSE RATING: " + df['courseratings_stars'].astype(str) + ". " +
    "JOB RATING: " + df['jobcareer_impact_stars'].astype(str) + ". " +
    "FEE: $" + df['full_course_fee'].astype(str) + ". " +
    "FEE AFTER SUBSIDIES: $" + df['course_fee_after_subsidies'].astype(str) + ". " +
    "TRAINING HOUR: " + df['number_of_hours'].astype(str) + " hours. " +
    "TRAINING COMMITMENT: " + df['training_commitment'].astype(str) + ". " +
    "DESCRIPTIONS: " + df['about_this_course'].astype(str) + ". " +   
    "SKILLS: " + df['what_you_learn'].astype(str)
)

X = df['Full_Text'].values
y = df['attendancecount'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Important: Scale the target variable (y) for better regression performance
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

print(f"Sample Input Text:\n{X_train[0]}")
print(f"Sample Target Value (Original): {y_train[0]}")
print(f"Sample Target Value (Scaled): {y_train_scaled[0]}")


Sample Input Text:
TITLE: Java Programming for Complete Beginners(Classroom and Synchronous e-learning). SCHOOL: NON-IHL. COURSE RATING: 4.0. JOB RATING: 0.0. FEE: $1400.0. FEE AFTER SUBSIDIES: $420.0. TRAINING HOUR: 28.0 hours. TRAINING COMMITMENT: Part Time. DESCRIPTIONS: LO1: Gather and confi rm the architecture requirements_x000D_LO2: Derive high-level design from specifi cation_x000D_LO3: Apply Java language syntax and layout_x000D_LO4: Apply basic object-oriented interaction principles_x000D_LO5: Document the design. SKILLS: Object-oriented design is a process by which a set of detailed object-oriented software design models are built, which are then used by the programmers to write and test programs for the new system. Systems design is the bridge between user requirements and programming the new system. One strength of the object-oriented approach is that the design models are often an extension of the requirements models. Object-oriented design is an analytical, rigorous, and 

In [19]:
def build_custom_bert_regression_model():
    # Get the BERT Backbone and its Preprocessor
    backbone = keras_nlp.models.BertBackbone.from_preset(
        "bert_tiny_en_uncased",
        load_weights=True,
    )
    
    # Get the Preprocessor config (Crucial for tokenizing input)
    preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
        "bert_tiny_en_uncased",
        sequence_length=128
    )

    # Create the input layer and flow
    # The preprocessor handles the [CLS], [SEP], padding, and tokenization.
    inputs = tf.keras.Input(shape=(), dtype=tf.string)
    
    # Apply preprocessor to raw text input
    x = preprocessor(inputs)
    
    # Pass tokenized data through the BERT backbone
    # output_mask_index=0 ensures we get the hidden state of the [CLS] token
    x = backbone(x)['pooled_output'] 
    
    # Simple Regression Head
    x = tf.keras.layers.Dropout(0.1)(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x) # Single linear output for regression

    # Build the final model
    model = tf.keras.Model(inputs, outputs)
    
    # Compile for Regression
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
    )
    
    return model

# Create the model
regression_model = build_custom_bert_regression_model()
print("\nModel Summary:")
regression_model.summary()


Model Summary:


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_text_classifi… │ [(None, 128),     │          0 │ input_layer_5[0]… │
│ (BertTextClassifie… │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 128),     │  4,385,920 │ bert_text_classi… │
│ (BertBackbone)      │ (None, 128, 128)] │            │ bert_text_classi… │
│                     │                   │            │ bert_text_classi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 128)       │          0 │ bert_backbone[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        129 │ dropout_17[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,386,049 (16.73 MB)

 Trainable params: 4,386,049 (16.73 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# --- Configuration ---
PRESET = "bert_tiny_en_uncased"
MAX_SEQUENCE_LENGTH = 128 # Your limit

# --- 1. Load the Tokenizer ---
# We use the raw tokenizer to see the length BEFORE padding/truncation
tokenizer = keras_nlp.models.BertTokenizer.from_preset(
    PRESET
)

# --- 2. Define a function to calculate token length ---
# This function calculates the number of tokens for a given text,
# *including* the automatically added [CLS] and [SEP] tokens.
def get_token_length(text):
    # Tokenize the text.
    token_ids = tokenizer.tokenize(text)
    # The length of the resulting tensor is the number of tokens
    return tf.shape(token_ids)[0].numpy()

# --- 3. Apply the function to your full dataset (X_train) ---
print("Calculating token lengths...")
# Use list comprehension for efficient processing
token_lengths = [get_token_length(text) for text in X_train]

# Convert to a numpy array for easy statistical calculation
token_lengths = np.array(token_lengths)

# --- 4. Analyze Results ---

# Calculate statistics
avg_length = np.mean(token_lengths)
max_length = np.max(token_lengths)
min_length = np.min(token_lengths)

# Calculate truncation metrics
# An input is truncated if its length is greater than the MAX_SEQUENCE_LENGTH
truncated_count = np.sum(token_lengths > MAX_SEQUENCE_LENGTH)
total_samples = len(X_train)
percent_truncated = (truncated_count / total_samples) * 100

# --- 5. Print Summary ---
print("\n--- Token Length Analysis Summary ---")
print(f"BERT Preset Used: {PRESET}")
print(f"Max Sequence Length in Model: {MAX_SEQUENCE_LENGTH}")
print("-" * 40)
print(f"Total Training Samples: {total_samples}")
print(f"Average Token Length: {avg_length:.2f} tokens")
print(f"Maximum Token Length: {max_length} tokens")
print(f"Minimum Token Length: {min_length} tokens")
print("-" * 40)
print(f"Samples Exceeding {MAX_SEQUENCE_LENGTH} Tokens (Truncated): {truncated_count} samples")
print(f"Percentage of Truncated Samples: {percent_truncated:.2f}%")
print("-" * 40)

# Example of a long and short text's length
long_sample_index = np.argmax(token_lengths)
short_sample_index = np.argmin(token_lengths)

print(f"Length of Longest Sample: {token_lengths[long_sample_index]}")
print(f"Length of Shortest Sample: {token_lengths[short_sample_index]}")

# Note: The calculation above includes the [CLS] and [SEP] tokens.

Calculating token lengths...

--- Token Length Analysis Summary ---
BERT Preset Used: bert_tiny_en_uncased
Max Sequence Length in Model: 64
----------------------------------------
Total Training Samples: 20650
Average Token Length: 296.29 tokens
Maximum Token Length: 1426 tokens
Minimum Token Length: 66 tokens
----------------------------------------
Samples Exceeding 64 Tokens (Truncated): 20650 samples
Percentage of Truncated Samples: 100.00%
----------------------------------------
Length of Longest Sample: 1426
Length of Shortest Sample: 66


<h3>Train BERT Model</h3>

In [20]:
early_stopping = EarlyStopping(
    monitor='val_mae', 
    patience=1, 
    restore_best_weights=True,
    verbose=1 
)

history = regression_model.fit(
    x=X_train,
    y=y_train_scaled,
    validation_data=(X_test, y_test_scaled),
    epochs=3,
    batch_size=4, #change from 16 to 4 as WSL keep crashing during training
    callbacks=[early_stopping]
)

# Make predictions on the test set (results are scaled)
y_pred_scaled = regression_model.predict(X_test)

# Inverse transform the predictions to the original student count scale
y_pred = scaler.inverse_transform(y_pred_scaled)

# Display results
results_df = pd.DataFrame({
    'Actual_Students': y_test,
    'Predicted_Students': y_pred.flatten().round(0) # Round to nearest whole number
})

print("\n--- Sample Predictions (Test Set) ---")
print(results_df)

# Calculate final unscaled MAE
final_mae = np.mean(np.abs(results_df['Actual_Students'] - results_df['Predicted_Students']))
print(f"\nFinal Mean Absolute Error (Unscaled): {final_mae:.2f} students")

Epoch 1/3


2025-11-27 11:05:46.566601: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


5163/5163 ━━━━━━━━━━━━━━━━━━━━ 257s 48ms/step - loss: 1.0302 - mae: 0.4131 - val_loss: 0.7664 - val_mae: 0.2466
Epoch 2/3
5163/5163 ━━━━━━━━━━━━━━━━━━━━ 251s 49ms/step - loss: 0.9279 - mae: 0.3553 - val_loss: 0.7416 - val_mae: 0.2313
Epoch 3/3
5163/5163 ━━━━━━━━━━━━━━━━━━━━ 251s 49ms/step - loss: 0.8208 - mae: 0.3110 - val_loss: 0.7468 - val_mae: 0.2399
Epoch 3: early stopping
Restoring model weights from the end of the best epoch: 2.
162/162 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step

--- Sample Predictions (Test Set) ---
      Actual_Students  Predicted_Students
0                 0.0               -73.0
1                 0.0               -54.0
2              1371.0               623.0
3               436.0               205.0
4                32.0                28.0
...               ...                 ...
5158             27.0                40.0
5159              5.0               -30.0
5160             22.0               159.0
5161             13.0               -74.0
5162             

In [21]:
# R2 Score (Coefficient of Determination)
# Best value is 1.0. Can be negative.
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2:.4f}")

# Mean Absolute Error (MAE)
# Measures the average magnitude of the errors in a set of predictions.
# Best value is 0.0.
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Mean Squared Error (MSE)
# Measures the average of the squares of the errors. Penalizes larger errors more.
# Best value is 0.0.
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

R2 Score: 0.1736
Mean Absolute Error (MAE): 169.0910
Mean Squared Error (MSE): 396445.7933


<h3>Save the model</h3>
<p>Note: Save the model immediately as it take 30 - 60mins to train a model</p>

In [25]:
#Save the mode. Remember to change model name if needed
MODEL_SAVE_PATH = 'data/model/bert_regression_model_vs2.keras'
regression_model.save(MODEL_SAVE_PATH) 
print(f"\nModel saved successfully to: {MODEL_SAVE_PATH}")


Model saved successfully to: data/model/bert_regression_model_vs2.keras


<h3>Load the model (for testing)</h3>

In [11]:
model_path = 'data/model/bert_regression_model_vs2.keras' 
loaded_model = keras.models.load_model(model_path)

ValueError: Exception encountered when calling BertTextClassifierPreprocessor.call().

[1mCould not automatically infer the output shape / dtype of 'bert_text_classifier_preprocessor_5' (of type BertTextClassifierPreprocessor). Either the `BertTextClassifierPreprocessor.call()` method is incorrect, or you need to implement the `BertTextClassifierPreprocessor.compute_output_spec() / compute_output_shape()` method. Error encountered:

Exception encountered when calling BertTokenizer.call().

[1mNo vocabulary has been set for WordPieceTokenizer. Make sure to pass a `vocabulary` argument when creating the layer.[0m

Arguments received by BertTokenizer.call():
  • inputs=tf.Tensor(shape=(None,), dtype=string)
  • args=<class 'inspect._empty'>
  • training=None
  • kwargs=<class 'inspect._empty'>[0m

Arguments received by BertTextClassifierPreprocessor.call():
  • args=('<KerasTensor shape=(None,), dtype=string, sparse=False, ragged=False, name=input_layer_5>',)
  • kwargs=<class 'inspec

In [ ]:
def predict_new_course_students(model, scaler, course_data):
    #Remeber to change the key to new one after testing the first model
    new_input_text = (
        "TITLE: " + str(course_data['coursetitle']) + ". " +
        "SCHOOL: " + str(course_data['ihl_status']) + ". " +
        "COURSE RATING: " + str(course_data['courseratings_stars']) + ". " +
        "JOB RATING: " + str(course_data['jobcareer_impact_stars']) + ". " +
        "FEE: $" + str(course_data['full_course_fee']) + ". " +
        "FEE AFTER SUBSIDIES: $" + str(course_data['course_fee_after_subsidies']) + ". " +
        "TRAINING HOUR: " + str(course_data['number_of_hours']) + " hours. " +
        "TRAINING COMMITMENT: " + str(course_data['training_commitment']) + ". " +
        "DESCRIPTIONS: " + str(course_data['about_this_course']) + ". " + 
        "SKILLS: " + str(course_data['what_you_learn'])
    )

    # Convert the single string into a NumPy array of strings for the model
    X_new = np.array([new_input_text])
    
    # --- B. Run Prediction ---
    # Model predicts the scaled student count
    y_pred_scaled = model.predict(X_new)
    
    # --- C. Inverse Transform and Finalize ---
    # Convert scaled value back to original student count
    y_pred = scaler.inverse_transform(y_pred_scaled)
    
    # Round to the nearest whole number and ensure it's not negative
    predicted_students = max(0, int(round(y_pred[0][0])))
    
    return predicted_students, new_input_text

# -----------------------------------------------------------------------------
## 🎯 Define the New Course Data
# -----------------------------------------------------------------------------

# Define the features for a hypothetical new course
new_course_features = {
    'coursetitle': "Engineering Ethics",
    'ihl_status': "IHL",
    'courseratings_stars': 4.0,
    'jobcareer_impact_stars': 4.0,
    'full_course_fee': 1461.0,
    'course_fee_after_subsidies': 438.30,
    'number_of_hours': 36,
    'training_commitment': "Full-time",
    'about_this_course': "Refer to website",
    'what_you_learn': "Refer to website"
    #'about_this_course': "Nowadays, technology has a pervasive and profound impact on everyday life, where engineers play a crucial role in its development. It is therefore extremely important that engineers understand the importance of safety, health, and welfare of the public, when developing this technology. They must be morally committed and equipped to tackle any ethical issues they may encounter. This course aims at training the student to reach such a status through discussions and typical case studies where real examples are thoroughly discussed.",
    #'what_you_learn': "Ethics and Professionalism. Moral Choices and Ethical Dilemmas. Codes of Ethics. Moral Frameworks. Ethics as Social Experimentation. Safety and Risk. Assessing and Reducing Risk. Workplace Responsibilities and Rights. Truth and Truthfulness. Computer Ethics. Environmental Ethics;"
}

# --- Execute the Prediction ---
if regression_model:
    predicted_count, input_text = predict_new_course_students(loaded_model, scaler, new_course_features)

    print("\n--- NEW COURSE PREDICTION ---")
    print(f"Predicted Number of Students to Attend: **{predicted_count}**")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step

--- NEW COURSE PREDICTION ---
Predicted Number of Students to Attend: **113**
